<a href="https://colab.research.google.com/github/lightfury36/Natural-Language-Processsing/blob/main/Fake_News_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import files
uploaded = files.upload ()

Saving train.csv to train.csv


In [17]:
df = pd.read_csv("/content/train.csv")

In [18]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
df=df.dropna()

In [20]:
x = df.drop("label", axis=1)

In [21]:
y=df["label"]

In [22]:
x.shape

(18285, 4)

In [23]:
y.shape

(18285,)

In [24]:
import tensorflow as tf

In [25]:
tf.__version__

'2.4.1'

In [28]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

##Prepocessing##

In [29]:
voc_size=5000

In [31]:
import re
import nltk
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
messages=x.copy()

In [36]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [37]:
messages.reset_index(inplace=True)

In [38]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [42]:
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review=re.sub('[^a-zA-z]', ' ', messages['title'][i])
    review=review.lower()
    review=review.split()

    review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [43]:
len(corpus)

18285

##One hot representation##

In [44]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
#onehot_repr

##Emedding representation##

In [47]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2157  956 1855]
 [   0    0    0 ...  764   78 3165]
 [   0    0    0 ...  139 2889  922]
 ...
 [   0    0    0 ... 3713 3116 1665]
 [   0    0    0 ... 4450 1601  806]
 [   0    0    0 ...  443 3913 1104]]


In [48]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3869,
       4697, 4961, 1485, 3989, 2703,  973, 2157,  956, 1855], dtype=int32)

In [49]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
len(embedded_docs),y.shape

(18285, (18285,))

In [56]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [57]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

##Model Training##

In [59]:
##Final Training

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.4824 - accuracy: 0.7432 - val_loss: 0.1956 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1325 - accuracy: 0.9492 - val_loss: 0.1928 - val_accuracy: 0.9195
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0866 - accuracy: 0.9670 - val_loss: 0.2111 - val_accuracy: 0.9120
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0530 - accuracy: 0.9838 - val_loss: 0.2608 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0322 - accuracy: 0.9896 - val_loss: 0.2913 - val_accuracy: 0.9072
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0251 - accuracy: 0.9929 - val_loss: 0.3368 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0171 - accuracy: 0.9956 - val_loss: 0.4112 - val_accuracy: 0.9128

##Adding Droputs##

In [60]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

##Perfomance metrics##

In [62]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [63]:
from sklearn.metrics import confusion_matrix


In [64]:
confusion_matrix(y_test,y_pred)

array([[2166, 1253],
       [ 546, 2070]])

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
accuracy_score(y_test,y_pred)

0.7019055509527755